<a href="https://colab.research.google.com/github/komo135/tradingrl/blob/master/dist_rl.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Google ドライブをマウントするには、このセルを実行してください。
from google.colab import drive
drive.mount('/content/drive')

%cd drive/My Drive
%load_ext Cython

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/My Drive


In [2]:
!pip install ta

In [0]:
from ape_x import *
import os
import multiprocessing as mp
import tensorflow as tf
import numpy as np
import time
import shutil
from multiprocessing import Pool
from functools import lru_cache

df = "audpred15.csv"

_ = shutil.copy("/content/drive/My Drive/"+df,"/content")
# sac_one_step.ckpt
# saver_path = "td3"
saver_path = "sac1"

restore = False

# if restore == True:
#   _ = shutil.copy("/content/drive/My Drive/" + saver_path + ".data-00000-of-00001","/content")
#   _ = shutil.copy("/content/drive/My Drive/" + saver_path + ".index","/content")
#   _ = shutil.copy("/content/drive/My Drive/checkpoint","/content")
  
# if restore == True:
#   _ = shutil.copy("/content/drive/My Drive/model/" + saver_path + ".data-00000-of-00001","/content")
#   _ = shutil.copy("/content/drive/My Drive/model/"+ saver_path + ".index","/content")
#   _ = shutil.copy("/content/drive/My Drive/model/checkpoint","/content")

output_size = 3
window_size = 30
size = 96
num_actor = 3
mem_size = size * num_actor * 5000
  
def actor_work(queues, num):
  sess = tf.compat.v1.Session()
  actor = Actor(sess, df, window_size, num, size, output_size, saver_path, restore)
  actor.run(10, queues, 10, 1000,100,24,n=2,step=1)

device='/device:GPU:0'
# device = '/cpu:0'

def leaner_work(queues):
        sess = tf.compat.v1.Session()
        leaner = Leaner(sess, df, window_size, output_size,mem_size, device, saver_path, restore)
        leaner.leaner(queues)

In [0]:
%cd /content
import concurrent.futures as futures
# 
executor = futures.ThreadPoolExecutor(max_workers=num_actor+1)
queue = mp.Queue()

ps = [executor.submit(leaner_work, queue)]

for i in range(num_actor):
  ps.append(executor.submit(actor_work,queue,i))

for p in ps:
  p.result()

/content


/content/drive/My Drive/memory.py:127: RuntimeWarning: divide by zero encountered in double_scalars
  max_weight = (p_min * n) ** (-self.PER_b)


action probability =  [0.7473684210526316, 0.11578947368421053, 0.13684210526315788]
buy =  0.8421052631578947  sell =  0.1578947368421053
trade accuracy =  0.3448275862068966
epoch: 1, total rewards: -122.000000, mean rewards: -30.821053
action probability =  [0.7473684210526316, 0.10526315789473684, 0.1473684210526316]
buy =  0.8842105263157894  sell =  0.11578947368421055
trade accuracy =  0.25806451612903225
epoch: 11, total rewards: -682.000000, mean rewards: -172.294737
action probability =  [0.7157894736842105, 0.1368421052631579, 0.1473684210526316]
buy =  0.8526315789473684  sell =  0.1473684210526316
trade accuracy =  0.18421052631578946
epoch: 21, total rewards: -817.000000, mean rewards: -206.400000
action probability =  [0.8210526315789474, 0.10526315789473684, 0.0736842105263158]
buy =  0.8842105263157894  sell =  0.11578947368421055
trade accuracy =  0.3076923076923077
epoch: 31, total rewards: -180.000000, mean rewards: -45.473684
action probability =  [0.73684210526315